In [1]:
"""- Insert FAERS AEOLUS concepts in sqlite table.
- Find which concepts form AEOLUS are not in current RxNorm release (The concepts used in the AEOLUS database are mapped to the RxNorm vocabulary. However, some of the RxNorm IDs used 
in AEOLUS publication have since been retired or remapped)
- Use RxNorm API to get current status of remapped compounds
- Where the status is alien or retired, add this as a comment in the database
- If remapped, update the concept id in rxnorm_concept column and move old id to the via_rxnorm_concept column
- Update the names of the remapped concepts
- Add Drugbank ID to concepts where directly available from RXNorm CONSO file
- Next, for concepts without a drugbank_id, see if the forms (salt relationships from RxNorm RELA file) retrieve some more mappings
- Update the rows with these 'forms of' salt form concepts such that the original ID (the salt form) is moved to via_rxnorm_concept column, and rxnorm_concept and rxnorm_name are updated to the parent concept, and add comment 'salt form' in rxnorm_comment columns
- Add Drugbank ID if available for the salt parent form concept
"""

"- Insert FAERS AEOLUS concepts in sqlite table.\n- Find which concepts form AEOLUS are not in current RxNorm release (The concepts used in the AEOLUS database are mapped to the RxNorm vocabulary. However, some of the RxNorm IDs used \nin AEOLUS publication have since been retired or remapped)\n- Use RxNorm API to get current status of remapped compounds\n- Where the status is alien or retired, add this as a comment in the database\n- If remapped, update the concept id in rxnorm_concept column and move old id to the via_rxnorm_concept column\n- Update the names of the remapped concepts\n- Add Drugbank ID to concepts where directly available from RXNorm CONSO file\n- Next, for concepts without a drugbank_id, see if the forms (salt relationships from RxNorm RELA file) retrieve some more mappings\n- Update the rows with these 'forms of' salt form concepts such that the original ID (the salt form) is moved to via_rxnorm_concept column, and rxnorm_concept and rxnorm_name are updated to the 

In [2]:
import requests
import json
import time
import sqlite3 as sqlite

In [26]:
# Set the following parameters:
basedir = '/Users/ines/FAERS_y2'
# Location of downloaded RxNorm files
rxnorm_conso_location = basedir + '/data/raw/RxNorm_full_03042019/rrf/RXNCONSO.RRF'
rxnorm_rela_location = basedir + '/data/raw/RxNorm_full_03042019/rrf/RXNREL.RRF'
# Location of the sqlite database used for the mapping process
mapping_process_db = basedir + '/data/interim/201903_drug_mapping_process.db'

In [4]:
# File with concepts occurring in FAERS AEOLUS database
with open(basedir + '/data/raw/faers_drug_concepts.txt') as f:
    aeolus_concepts = f.readlines()

In [5]:
len(aeolus_concepts)

3527

In [6]:
# Connect to database, empty at this moment
conn = sqlite.connect(mapping_process_db)
cur = conn.cursor()

In [7]:
# Insert FAERS AEOLUS concepts in sqlite table, and close db
rxnorm_aeolus = dict()

for row in aeolus_concepts[1:]:
    elements = row.strip('\n').split('\t')
    
    aeolus_concept = elements[0]
    rxnorm_name = elements[1]
    rxnorm_concept = elements[2]
    rxnorm_aeolus[rxnorm_concept] = {'rxnorm_name': rxnorm_name, 'aeolus_concept': aeolus_concept}
    cur.execute("""insert or ignore into drug_concepts(aeolus_concept, rxnorm_name, rxnorm_concept) values(?,?,?)""", (aeolus_concept, rxnorm_name, rxnorm_concept))
conn.commit()
conn.close()

In [8]:
# RxNorm version in the current RxNorm API # Ideally this is the same as the RxNorn downloaded files
response = requests.get('https://rxnav.nlm.nih.gov/REST/version.json')
response.content.decode()

'{"version":"04-Mar-2019"}'

In [9]:
# File RXNCONSO provided by RxNorm
with open(rxnorm_conso_location) as f:
    conso = f.readlines()

In [10]:
# Set up two sets of ids, one with all rxnorm concepts from CONSO and one with FAERS AEOLUS concepts
rxnorm_all = set()
for row in conso:
    elements = row.strip('\n').split('|')
    if elements[11] == 'RXNORM':
        rxnorm_all.add(int(elements[0]))
    else:
        next

aeolus_all = set()
for row in aeolus_concepts[1:]: 
    elements = row.strip('\n').split('\t')
    rxnorm_concept = elements[2]
    aeolus_all.add(int(rxnorm_concept))
    
# Number of concept ids have come from aeolus but are not in this rxnorm version
missing = aeolus_all - rxnorm_all
len(missing)

71

In [ ]:
%%time
# There are some rxnorm concepts that have been retired/remapped. Need to get the new id.
# Use RxNorm API to get their statuses or remapped compounds
# the RXNCUICHANGES file provided by RxNorm is not cumulative (only has latest version changes),
# but API provides status of remapped compounds, so that is useful.
# This takes some minutes

missing_statuses = {}
for id in list(missing):
    response = requests.get('https://rxnav.nlm.nih.gov/REST/rxcui/{}/status.json'.format(id))
    missing_statuses[id] = json.loads(response.content.decode('utf-8'))['rxcuiStatus']
    time.sleep(0.15)

In [14]:
# Open db again
conn = sqlite.connect(mapping_process_db)
cur = conn.cursor()

In [15]:
# Where the status is alien or retired, add this as a comment in the database
# If remapped, update the concept id in rxnorm_concept column and move old id to the via_rxnorm_concept column

for old_id in missing_statuses.keys():
    if missing_statuses[old_id]['status'] == 'Remapped':
        new_id = missing_statuses[old_id]['minConceptGroup']['minConcept'][0]['rxcui']
        cur.execute("update drug_concepts set rxnorm_concept = ?, rxnorm_comment='remapped', via_rxnorm_concept = ? where rxnorm_concept = {}".format(old_id), (new_id, old_id))
        
    if missing_statuses[old_id]['status'] == 'Retired':
        cur.execute("update drug_concepts set rxnorm_comment = 'retired' where rxnorm_concept = {}".format(old_id))

    if missing_statuses[old_id]['status'] == 'Alien':
        cur.execute("update drug_concepts set rxnorm_comment = 'alien' where rxnorm_concept = {}".format(old_id))

conn.commit()

In [16]:
# This should be an empty list, all these IDs should now be replaced
cur.execute("""select * from drug_concepts where rxnorm_concept in ({}) and rxnorm_comment not in ('alien', 'retired')""".format(', '.join([str(i) for i in missing]))).fetchall()

[]

In [17]:
# All missing concepts are accounted for
print('number of retired compounds: ' + str(cur.execute("""select count(*) from drug_concepts where rxnorm_comment = 'retired'""".format(', '.join([str(i) for i in missing]))).fetchall()))
print('number of alien compounds: ' + str(cur.execute("""select count(*) from drug_concepts where rxnorm_comment = 'alien'""".format(', '.join([str(i) for i in missing]))).fetchall()))
print('number of compounds remapped: '+ str(cur.execute("""select count(*) from drug_concepts where via_rxnorm_concept in ({})""".format(', '.join([str(i) for i in missing]))).fetchall()))

number of retired compounds: [(27,)]
number of alien compounds: [(37,)]
number of compounds remapped: [(7,)]


In [18]:
# Make a dictionary of current rxnorm names from CONSO file 

conso_names_dict = {}
for row in conso:
    elements = row.strip('\n').split('|')
    if elements[11] == 'RXNORM':
        conso_names_dict[elements[0]] = elements[14]
    else:
        next


In [19]:
# Update the names of the remapped concepts

for elem in [i[0] for i in cur.execute("""select rxnorm_concept from drug_concepts where rxnorm_comment = 'remapped'""").fetchall()]:
    new_name = conso_names_dict[str(elem)]
    cur.execute("""update drug_concepts set rxnorm_name = ? where rxnorm_concept = {}""".format(str(elem)), (new_name,))

In [20]:
conn.commit()
conn.close()

### Check RELA file for salt-parent relationships

In [21]:
conn = sqlite.connect(mapping_process_db)
cur = conn.cursor()

In [22]:
current_concepts = [i[0] for i in cur.execute("select distinct rxnorm_concept from drug_concepts where rxnorm_comment != 'alien' or rxnorm_comment is NULL").fetchall()]

In [23]:
len(current_concepts)

3484

In [24]:
current_concepts[:10]

[26744, 124427, 762595, 762817, 763100, 763656, 797635, 797752, 798232, 798266]

In [27]:
# File RXNREL provided by RxNorm
with open(rxnorm_rela_location) as f:
    rela = f.readlines()

In [28]:
# this reads over the REL table from RxNorm files and if the concept is among the current_concepts, it's added to the dictionary
# This REL table holds the relationships, the direction of (relationship): the REL is the relationship which the SECOND concept HAS TO the FIRST concept
# this takes a few minutes
salt_dict = []

for row in rela:
    if 'form_of' in row:
        elements = row.strip('\n').split('|')
        concept_1 = elements[0]
        concept_2 = elements[4]
        relationship = elements[7]
        
        if concept_2 in [str(i) for i in current_concepts] and relationship == 'form_of':
            salt_dict.append({'concept_1': concept_1, 'concept_2': concept_2, 'relationship': relationship})

In [29]:
len(salt_dict)

24

In [30]:
# Retrieve rows where the rxnorm_concept is the 'form of' (concept_2) of concept_1
# These rows need to be updated (see next)
for item in salt_dict:
    print(cur.execute("""select * from drug_concepts where rxnorm_concept = {}""".format(str(item['concept_2']))).fetchall())

[(40165377, 'abobotulinumtoxinA', 860168, None, None, None, None, None, None, None, None)]
[(40165651, 'onabotulinumtoxinA', 860189, None, None, None, None, None, None, None, None)]
[(40224763, 'incobotulinumtoxin A', 1000104, None, None, None, None, None, None, None, None)]
[(19055078, 'CLORAZEPIC ACID', 235408, None, None, None, None, None, None, None, None)]
[(1701677, 'colistimethate', 2708, None, None, None, None, None, None, None, None)]
[(19096877, 'Anhydrous Dextrose', 349730, None, None, None, None, None, None, None, None)]
[(19055137, 'heparin, porcine', 235473, None, None, None, None, None, None, None, None)]
[(46221354, 'menthol, (+)-', 1648123, None, None, None, None, None, None, None, None)]
[(19014218, 'SODIUM PHOSPHATE DIHYDRATE', 314837, None, None, None, None, None, None, None, None)]
[(19046939, 'Clavulanic Acid', 21216, None, None, None, None, None, None, None, None)]
[(19112899, 'ziconotide acetate', 486126, None, None, None, None, None, None, None, None)]
[(190110

In [31]:
# Update the rows with these 'forms of' such that the original ID (the salt form) is moved to via_rxnorm_concept column, and rxnorm_concept and rxnorm_name are updated to the parent concept

for salt_form in salt_dict:
    original_concept = salt_form['concept_2']
    parent_concept = salt_form['concept_1']
    parent_name = conso_names_dict[parent_concept]
    cur.execute("""update drug_concepts set rxnorm_concept = ?, rxnorm_name = ?, via_rxnorm_concept = ?, rxnorm_comment = 'salt form' where rxnorm_concept = {} and drugbank_id is null""".format(original_concept), (parent_concept, parent_name, original_concept))

In [32]:
# Just to confirm that the 'salt form' comment has not been placed on the previously existing concepts for the parent concepts
for item in salt_dict:
    print(cur.execute("""select * from drug_concepts where rxnorm_concept = {}""".format(str(item['concept_1']))).fetchall())

[(729855, 'Botulinum Toxin Type A', 1712, None, None, None, None, None, None, None, None), (40165377, 'Botulinum Toxin Type A', 1712, 860168, 'salt form', None, None, None, None, None, None), (40165651, 'Botulinum Toxin Type A', 1712, 860189, 'salt form', None, None, None, None, None, None), (40224763, 'Botulinum Toxin Type A', 1712, 1000104, 'salt form', None, None, None, None, None, None)]
[(729855, 'Botulinum Toxin Type A', 1712, None, None, None, None, None, None, None, None), (40165377, 'Botulinum Toxin Type A', 1712, 860168, 'salt form', None, None, None, None, None, None), (40165651, 'Botulinum Toxin Type A', 1712, 860189, 'salt form', None, None, None, None, None, None), (40224763, 'Botulinum Toxin Type A', 1712, 1000104, 'salt form', None, None, None, None, None, None)]
[(729855, 'Botulinum Toxin Type A', 1712, None, None, None, None, None, None, None, None), (40165377, 'Botulinum Toxin Type A', 1712, 860168, 'salt form', None, None, None, None, None, None), (40165651, 'Botuli

In [33]:
conn.commit()
conn.close()

### Next, let's find out which concepts have a Drugbank_id in the RxNorm CONSO files

In [34]:
# Load a dictionary with the rxnorm concepts that have a DrugBank ID in the RXNCONSO file mapped

rxnorm_drugbank = dict()

for row in conso:
    elements = row.strip('\n').split('|')
    if elements[11] == 'DRUGBANK':
        rxnorm_drugbank[elements[0]] = elements[13]
    else:
        next

In [35]:
len(rxnorm_drugbank)

7974

In [36]:
conn = sqlite.connect(mapping_process_db)
cur = conn.cursor()

In [37]:
current_rxnorm_ids = [i[0] for i in cur.execute("""select rxnorm_concept from drug_concepts where rxnorm_comment not in ('alien', 'retired') or rxnorm_comment is null""")]
print(current_rxnorm_ids[:10], '\n length is: ' + str(len(current_rxnorm_ids)))

[26744, 124427, 762595, 762817, 763100, 763656, 797635, 797752, 798232, 798266] 
 length is: 3462


In [38]:
# Update concepts in sqlite table where drugbank_id is directly available
for rxnorm_concept in current_rxnorm_ids:
    try:
        drugbank_id = rxnorm_drugbank[str(rxnorm_concept)]
        cur.execute("""update drug_concepts set drugbank_id = ? where rxnorm_concept = {}""".format(str(rxnorm_concept)), (drugbank_id,))
    except KeyError:
        pass
conn.commit()

In [39]:
# Number of concepts mapped to DB at this stage:
cur.execute("""select count(drugbank_id) from drug_concepts where drugbank_id is not null""").fetchall()

[(2472,)]

In [40]:
# Next, for concepts without a drugbank_id, how many are still missing
still_missing_drugbank = [i[0] for i in cur.execute("""select rxnorm_concept from drug_concepts where drugbank_id is null and (rxnorm_comment not in ('alien', 'retired') or rxnorm_comment is null) """)]
len(still_missing_drugbank)

990

In [41]:
conn.commit()
conn.close()